# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data.head(10)

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,City Date
0,0,East London,-33.0153,27.9116,61.92,81,16,4.50,ZA,1635649071
1,1,Dikson,73.5069,80.5464,6.19,90,94,15.08,RU,1635649074
2,2,Punta Arenas,-53.1500,-70.9167,42.91,70,0,4.61,CL,1635649075
3,3,Hirado,33.3597,129.5531,69.15,65,6,18.97,JP,1635649077
4,4,Carnarvon,-24.8667,113.6333,73.47,56,0,18.41,AU,1635648791
5,5,Provideniya,64.3833,-173.3000,12.74,73,88,11.56,RU,1635649081
6,6,Bredasdorp,-34.5322,20.0403,52.92,94,39,3.44,ZA,1635649083
7,7,Saint George,37.1041,-113.5841,65.66,47,100,1.99,US,1635649084
8,8,Kapaa,22.0752,-159.3190,73.90,76,90,5.75,US,1635649086
9,9,Hermanus,-34.4187,19.2345,59.81,88,78,8.32,ZA,1635649088


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Fill NaN values and convert to float
humidity = city_data["Humidity"].astype(float)

In [5]:
# Customize the size of the figure
# Plot Heatmap
fig = gmaps.figure(center = (40.7 , 74), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#my prefered weather criteria: temp under 65, humity less than 30, wind speed under 15
hotel_df = city_data.loc[(city_data['Temperature'] <65) & (city_data['Humidity'] <30) & (city_data['Wind Speed'] <15)]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,City Date
170,170,Changping,40.2186,116.2197,59.25,29,3,7.00,CN,1635649318
516,516,Safford,32.8340,-109.7076,63.52,29,1,8.05,US,1635649807
568,568,Manzhouli,49.6000,117.4333,34.36,23,1,4.25,CN,1635649880


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_name = hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\brand\AppData\Local\Temp/ipykernel_8608/3001886289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_name = hotel_df["Hotel Name"] = ""


,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,City Date,Hotel Name
170,170,Changping,40.2186,116.2197,59.25,29,3,7.00,CN,1635649318,
516,516,Safford,32.8340,-109.7076,63.52,29,1,8.05,US,1635649807,
568,568,Manzhouli,49.6000,117.4333,34.36,23,1,4.25,CN,1635649880,


In [8]:
#loop over to grab geocoordinates
hotel = []
for index,row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    coordinates = f"{lat},{lng}"
    target_search = "lodging"
    target_radius = 5000
    
# set up a parameters dictionary
    params = {"location": coordinates,
              "keyword": target_search, 
              "radius": target_radius,  
              "key": g_key
             }

# # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # run a request using our params dictionary and append hotel name into empty list
    response = requests.get(base_url, params=params)
    hotel_response = response.json()
   
    try:
        print(hotel_response["results"][0]["name"])
        hotel.append(hotel_response["results"][0]["name"])
    except:
        print("Cant' locate hotel.")
    

中国政法大学国际交流中心
Best Western Desert Inn
Friendship Hotel - Manzhouli


In [9]:
hotel_df["Hotel Name"] = hotel
hotel_df

C:\Users\brand\AppData\Local\Temp/ipykernel_8608/3005990346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel


,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,City Date,Hotel Name
170,170,Changping,40.2186,116.2197,59.25,29,3,7.00,CN,1635649318,中国政法大学国际交流中心
516,516,Safford,32.8340,-109.7076,63.52,29,1,8.05,US,1635649807,Best Western Desert Inn
568,568,Manzhouli,49.6000,117.4333,34.36,23,1,4.25,CN,1635649880,Friendship Hotel - Manzhouli


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig
#embed_minimal_html('map2.html', views=[fig])

Figure(layout=FigureLayout(height='420px'))